# load data

In [3]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import data_utils as D
with open('./squad/train-v1.1.json') as f:
    squad = json.load(f)
pair, _ = D.format_data(squad)
TOKENIZER_RE = D.TOKENIZER_RE

In [ ]:
pairs, _ = D.format_data(squad)
v = D.create_vocab(pairs)
with open('./squad/vocab_%d.json'%len(v),'w') as f:
    json.dump(v,f)

In [46]:
def conform(a,b,i,j):
    if i == len(a):
        return True
    if j == len(b):
        return False
    if a[i] != b[j]:
        return False
    if a[i] == b[j]:
        return conform(a,b,i+1,j+1)
    
def locate_answer(plist,alist):
    shape = [alist.shape[1]+1, plist.shape[1]+1] # N,T+1,Q+1
    answers = []
    for n,spa in enumerate(zip(plist, alist)):
#         print n, spa
        idx = []
        sp, sa = spa
        i_a =0; i_p = 0
        while i_a < len(sa) and sa[i_a]!=0: i_a +=1
        while i_p < len(sp) and sp[i_p]!=0: i_p +=1
        sa = sa[:i_a]; sp = sp[:i_p]
        start = []
        for i in range(len(sp)):
            if conform(sa, sp, 0, i):
                start.append(i)
#         print start
        for t in range(i_a):
            idx += [ [t,s+t] for s in start]
        idx.append( [i_a,shape[1]-1] )
        value = [1]*len(idx)
        answers.append([idx,value,shape])
    return answers
        
# tmpa = .copy()    
# locate_answer(p[:2],tmpa[:2])

def _transform_string(s,v,length):
    unk = v['<UNK>']
    Bian = np.zeros(length)
    for i, t in enumerate(s):
        Bian[i] = v[t] if t in v else unk
    return Bian

def transform(voca_path, data, qLen=None, pLen=None, aLen=None, shuffle=True):
    triple=[]
    ql = 0
    pl = 0
    al = 0
    for p, qas in data:
        p = TOKENIZER_RE.findall(p)
        if len(p) > pl: pl = len(p)
        for q,a in qas:
            q = TOKENIZER_RE.findall(q)
            a = TOKENIZER_RE.findall(a)
            if len(q)>ql: ql=len(q)
            if len(a)>al: al=len(a)
            triple.append([p,q,a])
                
    if qLen is None or ql<qLen: qLen = ql
    if pLen is None or pl<pLen: pLen = pl
    if aLen is None or al<aLen: aLen = al
    with open(voca_path, 'r') as f:
        voca = json.load(f)
    q_np = []
    p_np = []
    a_np = []
    for p,q,a in triple:
        p_np.append(_transform_string(p,voca,pLen))
        q_np.append(_transform_string(q,voca,qLen))
        a_np.append(_transform_string(a,voca,aLen))
    p_np = np.stack(p_np).astype(np.int)
    q_np = np.stack(q_np).astype(np.int)
    a_np = np.stack(a_np).astype(np.int)
    
#     locate_answer(p_np,a_np)
    return p_np, q_np, a_np

In [43]:
p,q,a = transform('./squad/vocab_99036.json', pair)
print p.shape, q.shape, a.shape

(87599, 671) (87599, 40) (87599, 43)


In [48]:
ans = locate_answer(p,a)

In [52]:
for _ in ans[:50]: print _

[[[0, 90], [1, 91], [2, 92], [3, 671]], [1, 1, 1, 1], [44, 672]]
[[[0, 32], [1, 33], [2, 34], [3, 35], [4, 36], [5, 671]], [1, 1, 1, 1, 1, 1], [44, 672]]
[[[0, 25], [0, 49], [1, 26], [1, 50], [2, 27], [2, 51], [3, 671]], [1, 1, 1, 1, 1, 1, 1], [44, 672]]
[[[0, 66], [1, 67], [2, 68], [3, 69], [4, 70], [5, 71], [6, 72], [7, 671]], [1, 1, 1, 1, 1, 1, 1, 1], [44, 672]]
[[[0, 14], [1, 15], [2, 16], [3, 17], [4, 18], [5, 19], [6, 20], [7, 671]], [1, 1, 1, 1, 1, 1, 1, 1], [44, 672]]
[[[0, 39], [1, 40], [2, 671]], [1, 1, 1], [44, 672]]
[[[0, 46], [0, 68], [1, 671]], [1, 1, 1], [44, 672]]
[[[0, 91], [0, 119], [0, 146], [0, 190], [1, 92], [1, 120], [1, 147], [1, 191], [2, 671]], [1, 1, 1, 1, 1, 1, 1, 1, 1], [44, 672]]
[[[0, 20], [0, 194], [1, 671]], [1, 1, 1], [44, 672]]
[[[0, 140], [1, 671]], [1, 1], [44, 672]]
[[[0, 20], [1, 671]], [1, 1], [44, 672]]
[[[0, 24], [0, 85], [0, 104], [1, 25], [1, 86], [1, 105], [2, 671]], [1, 1, 1, 1, 1, 1, 1], [44, 672]]
[[[0, 39], [1, 40], [2, 671]], [1, 1, 1], 

In [17]:
print p[:1]
print a[:1]
print idx, value, shape

[[97116 65170 58432   570 41593 86003 31303 39078 65170 44041 72719 75909
  87673 18776 41593 49353 10338 22830 65170 84017 69180 80035 90920 92166
  22830 65170 44041 52256 47792 32141 18779 18776 41593 82029 10338 22830
  95968  1405 62202 35961  1405 65170 75620 94742 13561 31508 64056 51668
  75542 65170 44041 52256 18776 65170 31436 22830 65170  7626 77964 80035
  68924 65170 31436 18776 65170   460 41593 48373 84372 22830 53019 47792
  18077 18779 18776 41593 39868 22830 65170   460 13577 69602 13796 38079
  65170 84017 69180 50690 14949 75542  1778 10801 68092 90920 53666 13577
  65170 21569 22830 65170 44041 56047 47792 90920 41593  9052 63793 91118
  42992 57041 50388  8694 47792 65170 75909 87673 18776 41593 82247 80575
  57918 10338 22830 69180     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [30]:
from scipy import sparse
i,v,s = ans[0]
print v, i, s
r = [_[0] for _ in i]
c = [_[1] for _ in i]
s = sparse.csc_matrix((v, (r,c)), s)

[1, 1, 1, 1] [[0, 90], [1, 91], [2, 92], [3, 671]] [44, 672]


In [32]:
print s.todense()

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [ ]:
def maxx(l):
    m = 0
    s = None
    for _ in l:
        if len(_) > m: m=len(_); s=_
    return m, s

In [ ]:
part = [ _[2] for _ in tri ]
m,s= maxx(part)
print m,s

## Test Graph

In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import tensorflow as tf
from match_LSTM import MatchLSTM
g = tf.Graph()
sess = tf.Session()

In [ ]:
batch_size = 10
vocab_size = 99036
q_length = q.shape[1]
p_length = p.shape[1]
print q_length, p_length
model = MatchLSTM(vocab_size,p_length, q_length, batch_size)
model.build_model()

In [ ]:
init = tf.initialize_all_variables()
sess.run(init)

In [ ]:
answer = sess.run( model.answer, 
                    feed_dict={ model.passage: p[:batch_size], model.question:q[:batch_size] } )

In [ ]:
answer.shape